<a href="https://colab.research.google.com/github/fabiobasson/Bi-Master/blob/main/geological_image_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import zipfile
import shutil
import numpy as np
import glob
import random
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

In [1]:
!rm -rf geological_similarity

**Importar arquivos de configuração do Kaglle**

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fabiobasson","key":"d71cbfdfd23379909173204eac74cf96"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

###  Download dos dados / descompactando os dados

In [5]:
!kaggle datasets download tanyadayanand/geological-image-similarity
! unzip -qq  geological-image-similarity

 51% 17.0M/33.6M [00:00<00:00, 20.5MB/s]
100% 33.6M/33.6M [00:01<00:00, 34.6MB/s]


In [ ]:
#!kaggle datasets download fabiobasson/geological-similarity
#! unzip -qq  geological-similarity

geological-similarity.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#! rm -rf gneiss andesite

### Data Preprocessing

In [6]:
andesite_dir = glob.glob('geological_similarity/andesite/*.jpg');gneiss_dir = glob.glob('geological_similarity/gneiss/*.jpg')
marble_dir = glob.glob('geological_similarity/marble/*.jpg');quartzite_dir = glob.glob('geological_similarity/quartzite/*.jpg')
rhyolite_dir = glob.glob('geological_similarity/rhyolite/*.jpg');
schist_dir = glob.glob('geological_similarity/schist/*.jpg')
print(len(andesite_dir));print(len(gneiss_dir)); print(len(marble_dir)); print(len(quartzite_dir));  print(len(quartzite_dir)); print(len(schist_dir))


5000
5000
4998
5000
5000
5000


In [7]:
andesite_dir[0:4]

['geological_similarity/andesite/9FUK8.jpg',
 'geological_similarity/andesite/20XR8.jpg',
 'geological_similarity/andesite/PT0WZ.jpg',
 'geological_similarity/andesite/Z7KXW.jpg']

In [ ]:
#geological_similarity = 'geological_similarity'
#subfolders = [f.path for f in os.scandir(geological_similarity) if f.is_dir()]


In [8]:
class_names = ['andesite', 'gneiss', 'marbler', 'quartzite', 'rhyolite','schist']
class_names[0]

'andesite'

In [9]:
andesite_df=[];gneiss_df=[];marble_df=[]
quartzite_df=[]
rhyolite_df=[]
schist_dir=[]

for i in andesite_dir:
    andesite_df.append([i,class_names[0]])
for j in gneiss_dir:
   gneiss_df.append([j,class_names[1]])
for l in marble_dir:
    marble_df.append([l,class_names[2]])
for m in quartzite_dir:
    quartzite_df.append([m,class_names[3]])
for n in rhyolite_dir:
    rhyolite_df.append([n,class_names[4]])
for o in schist_dir:
    schist_df.append([o,class_names[5]])
   
df = andesite_df + gneiss_df + marble_df + quartzite_df + rhyolite_df + schist_df
random.shuffle(df)
df

['geological_similarity/rhyolite/4RUEV.jpg',
 ['geological_similarity/marble/J4D0J.jpg', 'marbler'],
 ['geological_similarity/gneiss/HNN73.jpg', 'gneiss'],
 ['geological_similarity/andesite/DK8R1.jpg', 'andesite'],
 'geological_similarity/quartzite/BM7BI.jpg',
 ['geological_similarity/gneiss/ZRI8M.jpg', 'gneiss'],
 ['geological_similarity/andesite/AO6MW.jpg', 'andesite'],
 'geological_similarity/quartzite/SN3QA.jpg',
 ['geological_similarity/andesite/34CRV.jpg', 'andesite'],
 'geological_similarity/rhyolite/OD4F6.jpg',
 'geological_similarity/quartzite/AC63V.jpg',
 'geological_similarity/quartzite/6ILGR.jpg',
 ['geological_similarity/marble/P04EF.jpg', 'marbler'],
 ['geological_similarity/marble/6FQ0Z.jpg', 'marbler'],
 ['geological_similarity/marble/3F0N1.jpg', 'marbler'],
 'geological_similarity/rhyolite/4BV04.jpg',
 'geological_similarity/rhyolite/97STF.jpg',
 'geological_similarity/rhyolite/VHW87.jpg',
 'geological_similarity/rhyolite/07ZZS.jpg',
 ['geological_similarity/gneiss/3ZM

In [13]:
data_df = pd.DataFrame(df,columns=['path'])
data_df

,path
0,geological_similarity/rhyolite/4RUEV.jpg
1,"[geological_similarity/marble/J4D0J.jpg, marbler]"
2,"[geological_similarity/gneiss/HNN73.jpg, gneiss]"
3,"[geological_similarity/andesite/DK8R1.jpg, and..."
4,geological_similarity/quartzite/BM7BI.jpg
...,...
24993,"[geological_similarity/marble/MSTM9.jpg, marbler]"
24994,geological_similarity/quartzite/SR2CH.jpg
24995,geological_similarity/quartzite/CQTE4.jpg
24996,"[geological_similarity/andesite/AX1BA.jpg, and..."


In [ ]:
data_df.columns

RangeIndex(start=0, stop=18, step=1)